### PASO 1: Variables de configuracion: rutas a landing, raw y bronze

In [0]:
import io
import numpy as np
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, regexp_extract
from PIL import Image

In [0]:
account = spark.conf.get("adls.account.name")

landing_container = f"abfss://landing@{account}.dfs.core.windows.net"
lakehouse_container = f"abfss://lakehouse@{account}.dfs.core.windows.net"

landing_path = landing_container
raw_path = f"{lakehouse_container}/raw"
bronze_path = f"{lakehouse_container}/bronze"

print(landing_path)
print(raw_path)
print(bronze_path)

### PASO 2: Definimos funcion de utilidad que muestras todos los ficheros de un directorio

Si hay subdirectorios, incluye los ficheros presenetes en estos.

In [0]:
def list_all_files(path:str):
    """
    Lista recursivamente todos los archivos en una ruta y sus subcarpetas utilizando dbutils.fs.
    
    Parámetros:
        path (str): Ruta base en el sistema de archivos (e.g., 'dbfs:/mnt/mi_carpeta').
    
    Retorna:
        list: Lista de rutas completas de todos los archivos encontrados.
    """
    files_list = []
    try:
        # Listar contenidos de la ruta actual
        items = dbutils.fs.ls(path)
        
        for item in items:
            # Si es un archivo, añadirlo a la lista
            if item.isFile():
                files_list.append(item.path)
            # Si es un directorio, explorar recursivamente
            elif item.isDir():
                files_list.extend(list_all_files(item.path))
    except Exception as e:
        print(f"Error al listar {path}: {str(e)}")
    
    return files_list

### PASO 3: Creamos una clase que permite leer los ficheros de landing, para tipos de ficheros tanto json como jpeg

In [0]:

# ANTIGUO
def extract_label(path_col):
    """Extract label from file path using built-in SQL functions."""
    return regexp_extract(path_col, "flower_photos/([^/]+)", 1)
    
   
def extract_size(content):
    """Extract image size from its raw content."""
    image = Image.open(io.BytesIO(content))
    return image.size
    
    
@pandas_udf("width: int, height: int")
def extract_size_udf(content_series):
    sizes = content_series.apply(LandingStreamReader.extract_size)
    return pd.DataFrame(list(sizes))

In [0]:
# ANTIGUO
from pyspark.sql.functions import current_timestamp, input_file_name, replace,lit

class LandingStreamReader:

    IMAGE_EXTENSIONS = [
        "jpg", "jpeg", "png", "bmp", "gif", "tiff", "tif",
        "webp", "heic", "heif", "ico", "svg", "raw", "cr2",
        "nef", "orf", "arw", "psd", "indd", "ai", "eps"
    ]

    IMAGE_GENERAL_KEYWORD = "image"

    def __init__(self, builder):
        self.datasource = builder.datasource
        self.dataset = builder.dataset
        self.landing_path = builder.landing_path
        self.raw_path = builder.raw_path
        self.bronze_path = builder.bronze_path
        self.format = builder.format
        self.dataset_landing_path = f'{self.landing_path}/{self.datasource}/{self.dataset}'
        self.dataset_bronze_schema_location = f'{self.bronze_path}/{self.datasource}/{self.dataset}_schema'
        dbutils.fs.mkdirs(self.dataset_bronze_schema_location)
    
    def __str__(self):
        return (f"LandingStreamReader(datasource='{self.datasource}',dataset='{self.dataset}')")
        
    def add_metadata_columns(self,df):
        data_cols = df.columns
      
        metadata_cols = {
            "_ingested_at": current_timestamp(),
            "_ingested_filename": replace(
                                    input_file_name(),
                                    lit(self.landing_path),
                                    lit(self.raw_path) )
        }

        if (self.format in LandingStreamReader.IMAGE_EXTENSIONS) or (self.format == LandingStreamReader.IMAGE_GENERAL_KEYWORD):
            metadata_cols.update({
                "_size":extract_size_udf(col("content")),
                "_label":extract_label(col("path")),
            })

        for col_name, expr in metadata_cols.items():
            df = df.withColumn(col_name, expr) 
      
      #reordernamos columnas
        return df.select(list(metadata_cols.keys()) + data_cols)  
    
    def read_json(self):
      return (spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaLocation", self.dataset_bronze_schema_location)
            .load(self.dataset_landing_path)
        )

    def read_binary(self):
        return (spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format","binaryFile")
            # las imagenes vienen en subdirectorios con la etiqueta.
            .option("recursiveFileLookup", "true")
            .option("cloudFiles.schemaLocation", self.dataset_bronze_schema_location)
            .load(self.dataset_landing_path)

        )
   
    
    def read(self):
        
        df = None

        match self.format:
            case "json":
                df = self.read_json()
            case x if x in LandingStreamReader.IMAGE_EXTENSIONS or x == LandingStreamReader.IMAGE_GENERAL_KEYWORD:
                df = self.read_binary()
            case _:
                raise Exception(f"Format {self.format} not supported")

        if df:
            df = df.transform(self.add_metadata_columns)
            return df
    
    class Builder:
        def __init__(self):
            self.datasource = None
            self.dataset = None
            self.landing_path = None
            self.raw_path = None
            self.bronze_path = None
            self.format = None
        
        def set_datasource(self, datasource):
            self.datasource = datasource
            return self
        
        def set_dataset(self, dataset):
            self.dataset = dataset
            return self
        
        def set_landing_path(self, landing_path):
            self.landing_path = landing_path
            return self
        
        def set_raw_path(self, raw_path):
            self.raw_path = raw_path
            return self
        
        def set_bronze_path(self, bronze_path):
            self.bronze_path = bronze_path
            return self
          
        def set_format(self, format):
            self.format = format
            return self
        
        def build(self):
            return LandingStreamReader(self)
          


In [0]:
# Imports necesarios
import io
import pandas as pd
from PIL import Image
from pyspark.sql.functions import (
    col, lit, current_timestamp, input_file_name, replace, regexp_extract, pandas_udf
)

# Función auxiliar: extraer tamaño de imagen
def extract_size(content):
    image = Image.open(io.BytesIO(content))
    return image.size

# UDF para extraer (width, height)
@pandas_udf("width: int, height: int")
def extract_size_udf(content_series):
    sizes = content_series.apply(extract_size)
    return pd.DataFrame(list(sizes))

# Extraer etiqueta desde el path
def extract_label(path_col):
    return regexp_extract(path_col, "flower_photos/([^/]+)", 1)

# Añadir columnas de metadatos
def add_metadata_columns(df, landing_path, raw_path, format, image_extensions, image_keyword):
    data_cols = df.columns

    metadata_cols = {
        "_ingested_at": current_timestamp(),
        "_ingested_filename": replace(
            input_file_name(),
            lit(landing_path),
            lit(raw_path)
        )
    }

    if format in image_extensions or format == image_keyword:
        metadata_cols.update({
            "_size": extract_size_udf(col("content")),
            "_label": extract_label(col("path")),
        })

    for col_name, expr in metadata_cols.items():
        df = df.withColumn(col_name, expr)

    return df.select(list(metadata_cols.keys()) + [c for c in data_cols if c not in metadata_cols])

In [0]:
class LandingStreamReader:

    IMAGE_EXTENSIONS = [
        "jpg", "jpeg", "png", "bmp", "gif", "tiff", "tif",
        "webp", "heic", "heif", "ico", "svg", "raw", "cr2",
        "nef", "orf", "arw", "psd", "indd", "ai", "eps"
    ]

    IMAGE_GENERAL_KEYWORD = "image"

    def __init__(self, builder):
        self.datasource = builder.datasource
        self.dataset = builder.dataset
        self.landing_path = builder.landing_path
        self.raw_path = builder.raw_path
        self.bronze_path = builder.bronze_path
        self.format = builder.format
        self.dataset_landing_path = f'{self.landing_path}/{self.datasource}/{self.dataset}'
        self.dataset_bronze_schema_location = f'{self.bronze_path}/{self.datasource}/{self.dataset}_schema'
        dbutils.fs.mkdirs(self.dataset_bronze_schema_location)

    def __str__(self):
        return f"LandingStreamReader(datasource='{self.datasource}', dataset='{self.dataset}')"

    def read_json(self):
        return (spark.readStream
                .format("cloudFiles")
                .option("cloudFiles.format", "json")
                .option("cloudFiles.inferColumnTypes", "true")
                .option("cloudFiles.schemaLocation", self.dataset_bronze_schema_location)
                .load(self.dataset_landing_path))

    def read_binary(self):
        return (spark.readStream
                .format("cloudFiles")
                .option("cloudFiles.format", "binaryFile")
                .option("recursiveFileLookup", "true")
                .option("cloudFiles.schemaLocation", self.dataset_bronze_schema_location)
                .load(self.dataset_landing_path))

    def read(self):
        match self.format:
            case "json":
                df = self.read_json()
            case x if x in LandingStreamReader.IMAGE_EXTENSIONS or x == LandingStreamReader.IMAGE_GENERAL_KEYWORD:
                df = self.read_binary()
            case _:
                raise Exception(f"Format {self.format} not supported")

        return add_metadata_columns(
            df,
            self.landing_path,
            self.raw_path,
            self.format,
            LandingStreamReader.IMAGE_EXTENSIONS,
            LandingStreamReader.IMAGE_GENERAL_KEYWORD
        )

    class Builder:
        def __init__(self):
            self.datasource = None
            self.dataset = None
            self.landing_path = None
            self.raw_path = None
            self.bronze_path = None
            self.format = None

        def set_datasource(self, datasource):
            self.datasource = datasource
            return self

        def set_dataset(self, dataset):
            self.dataset = dataset
            return self

        def set_landing_path(self, landing_path):
            self.landing_path = landing_path
            return self

        def set_raw_path(self, raw_path):
            self.raw_path = raw_path
            return self

        def set_bronze_path(self, bronze_path):
            self.bronze_path = bronze_path
            return self

        def set_format(self, format):
            self.format = format
            return self

        def build(self):
            return LandingStreamReader(self)

### PASO 4: Clase de escritura, reutilizada del ejercicio anterior

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name, replace,lit

class BronzeStreamWriter:   
    def __init__(self, builder):
        self.datasource = builder.datasource
        self.dataset = builder.dataset
        self.landing_path = builder.landing_path
        self.raw_path = builder.raw_path
        self.bronze_path = builder.bronze_path
        self.dataset_landing_path = f"{self.landing_path}/{self.datasource}/{self.dataset}"
        self.dataset_raw_path =  f"{self.raw_path}/{self.datasource}/{self.dataset}"
        self.dataset_bronze_path = f"{self.bronze_path}/{self.datasource}/{self.dataset}"
        self.dataset_checkpoint_location = f'{dataset_bronze_path}_checkpoint'
        self.table = f'hive_metastore.bronze.{self.datasource}_{self.dataset}'
        self.query_name = f"bronze-{datasource}-{dataset}"
        dbutils.fs.mkdirs(self.dataset_raw_path)
        dbutils.fs.mkdirs(self.dataset_bronze_path)
        dbutils.fs.mkdirs(self.dataset_checkpoint_location)

    def __str__(self):
        return (f"BronzeStreamWriter(datasource='{self.datasource}',dataset='{self.dataset}')")
         
    def archive_raw_files(self,df):
      if "_ingested_filename" in df.columns:
        files = [row["_ingested_filename"] for row in df.select("_ingested_filename").distinct().collect()]
        for file in files:
          if file:
              file_landing_path = file.replace(self.dataset_raw_path,self.dataset_landing_path)
              dbutils.fs.mkdirs(file[0:file.rfind('/')+1])
              dbutils.fs.mv(file_landing_path,file)
    
    def write_data(self,df):
      spark.sql( 'CREATE DATABASE IF NOT EXISTS hive_metastore.bronze') 
      spark.sql(f"CREATE TABLE IF NOT EXISTS {self.table} USING DELTA LOCATION '{self.dataset_bronze_path}' ") 
      (df.write
          .format("delta")  
          .mode("append")
          .option("mergeSchema", "true")
          .option("path", self.dataset_bronze_path)
          .saveAsTable(self.table)
      )
        
    def append_2_bronze(self,batch_df, batch_id):
      batch_df.persist()
      self.write_data(batch_df)
      self.archive_raw_files(batch_df)
      batch_df.unpersist()
      

    class Builder:
        def __init__(self):
            self.datasource = None
            self.dataset = None
            self.landing_path = None
            self.raw_path = None
            self.bronze_path = None
        
        def set_datasource(self, datasource):
            self.datasource = datasource
            return self
        
        def set_dataset(self, dataset):
            self.dataset = dataset
            return self
        
        def set_landing_path(self, landing_path):
            self.landing_path = landing_path
            return self
        
        def set_raw_path(self, raw_path):
            self.raw_path = raw_path
            return self
        
        def set_bronze_path(self, bronze_path):
            self.bronze_path = bronze_path
            return self
        
        def build(self):
            return BronzeStreamWriter(self)

In [0]:
format = "jpg"
datasource = 'tensorflow'
dataset = "flower_photos"

#format = "json"
#datasource = 'retail'
#dataset = "sales_orders"

dataset_landing_path = f"{landing_path}/{datasource}/{dataset}"
dataset_raw_path =  f"{raw_path}/{datasource}/{dataset}"
dataset_bronze_path = f"{bronze_path}/{datasource}/{dataset}"

print(dataset_landing_path)
print(dataset_raw_path)
print(dataset_bronze_path)

In [0]:
reader = (LandingStreamReader.Builder()          
  .set_datasource(datasource)
  .set_dataset(dataset)
  .set_landing_path(landing_path)
  .set_raw_path(raw_path)
  .set_bronze_path(bronze_path)
  .set_format(format)
  .build()
)

print(reader)   

In [0]:
#display(reader.read().limit(100))

In [0]:
writer = (BronzeStreamWriter.Builder()
  .set_datasource(datasource)
  .set_dataset(dataset)
  .set_landing_path(landing_path)
  .set_raw_path(raw_path)
  .set_bronze_path(bronze_path)
  .build()
)

print(writer)

In [0]:
(reader
  .read()
  .writeStream
  .foreachBatch(writer.append_2_bronze)
  .trigger(availableNow=True)
  #.trigger(processingTime="60 seconds") # modo continuo
  .option("checkpointLocation", writer.dataset_checkpoint_location)
  .queryName(writer.query_name)
  .start()
)

In [0]:
query = f"""
select * 
from delta.`{writer.dataset_bronze_path}`
order by _ingested_at desc
limit 10
"""
display(spark.sql(query))

In [0]:
query = f"""
select distinct _ingested_filename 
from delta.`{writer.dataset_bronze_path}`
"""
display(spark.sql(query))